<a href="https://colab.research.google.com/github/greapis/jaeha-shim/blob/main/Building_vision_model_by_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **학습할 내용**


의류 기사를 인식하도록 신경망 학습시키기


네트워크의 다양한 레이어를 실험하는 과정을 안내하는 일련의 연습을 완료합니다.

## **빌드할 항목**

의류 물품을 식별하는 신경망

In [3]:
import tensorflow as tf
print(tf.__version__)

2.15.0


tf.keras.datasets MAPI에서는 Fashion MNIST 데이터를 제공합니다. 다음과 같이 로드합니다.

In [6]:
mnist = tf.keras.datasets.fashion_mnist

이 객체에서 load_data를 호출하면 두 가지 목록 모음인 학습 값과 테스트 값(의류 항목과 라벨을 표시하는 그래픽을 나타냄)을 제공합니다.

In [10]:
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

각 값의 모양 학습 이미지와 학습 라벨을 인쇄하여 확인합니다. 배열에서 다양한 색인을 실험해 볼 수 있습니다.